In [48]:
!gdown 'https://camtagstoreaiem.blob.core.windows.net/carb/best525.pt'

Downloading...
From: https://camtagstoreaiem.blob.core.windows.net/carb/best525.pt
To: /content/best525.pt
100% 22.5M/22.5M [00:03<00:00, 6.40MB/s]


In [49]:
!pip install -q gdown
%cd {HOME}
!gdown 'https://camtagstoreaiem.blob.core.windows.net/carb/20231023_195750.mp4'
!gdown 'https://camtagstoreaiem.blob.core.windows.net/carb/20231016_165314.mp4'

/content
Downloading...
From: https://camtagstoreaiem.blob.core.windows.net/carb/20231023_195750.mp4
To: /content/20231023_195750.mp4
100% 34.4M/34.4M [00:04<00:00, 7.88MB/s]
Downloading...
From: https://camtagstoreaiem.blob.core.windows.net/carb/20231016_165314.mp4
To: /content/20231016_165314.mp4
100% 32.5M/32.5M [00:04<00:00, 7.25MB/s]


In [62]:
SOURCE_VIDEO_PATH = f"{HOME}/20231016_165314.mp4"

In [63]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.203 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.2/166.8 GB disk)


In [52]:
!pip install supervision

from IPython import display
display.clear_output()

import supervision as sv

In [53]:
MODEL = "best525.pt"

In [54]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

Model summary (fused): 168 layers, 11127132 parameters, 0 gradients, 28.4 GFLOPs


In [55]:
print(model.model.names)

{0: 'bus', 1: 'car', 2: 'motorcycle', 3: 'truck'}


In [56]:
CLASS_NAMES_DICT = model.model.names
#selected_classes = [2, 3, 5, 7]

In [57]:
import supervision as sv
import numpy as np

In [64]:
LINE_START = sv.Point(50, 1500)
LINE_END = sv.Point(1030, 1500)

TARGET_VIDEO_PATH = f"{HOME}/vehicle-counting-result-with-counter.mp4"


In [65]:
sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

VideoInfo(width=1080, height=2400, fps=30, total_frames=385)

In [66]:
byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
trace_annotator = sv.TraceAnnotator(thickness=4, trace_length=50)
line_zone_annotator = sv.LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=2)
def callback(frame: np.ndarray, index:int) -> np.ndarray:
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = detections[np.isin(detections.class_id,[0,1,2,3])]
    detections = byte_tracker.update_with_detections(detections)
    labels = [
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for _, _, confidence, class_id, tracker_id
        in detections
    ]
    annotated_frame = trace_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )
    annotated_frame=box_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels)
    line_zone.trigger(detections)
    return  line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)
sv.process_video(
    source_path = SOURCE_VIDEO_PATH,
    target_path = TARGET_VIDEO_PATH,
    callback=callback
)